In [27]:
import os
import datetime as dt
import pandas as pd


In [28]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"
labelled_data_fnames = os.listdir(labelled_data_dir)
master_data_location  = f"{repo_dir}/data/labelled_master_data_{dt.date.today()}.csv"
print(labelled_data_fnames)
print(master_data_location)

['gm2858_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-09.csv', 'ac4119_labelling_assignment_2022-04-09.csv', 'yj2369_labelling_assignment_2022-04-09.csv']
/Users/ameliachu/repos/nlu-reddit-toxicity-dataset/data/labelled_master_data_2022-04-14.csv


In [29]:
fname

'ac4119_labelling_assignment_2022-04-09.csv'

In [30]:
 pd.read_csv(f"{labelled_data_dir}{fname}")

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat
0,100723,Jay woke up and chose violence,🖐,Always has been,0,0,0,0,0,0.0
1,108629,$SKT puts are PRINTING,When it hits $5k will you sell?,"Your username says it all, SAD",0,0,0,0,0,0.0
2,108801,Apples always a buy not advice,Diamond hands bro - relax,buy the dips,0,0,0,0,0,0.0
3,111247,It’s only been a couple weeks. Could easily cr...,About as good as time as any.,GME spike up?,0,0,0,0,0,0.0
4,113250,It's gonna be an ugly cry. My ass has never be...,I really appreciate that. Really. I’m spiral...,this was a really bad week to quit smoking,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
195,8613,Thanks guys just double checking,Apes together strong,That cracked me up. Investing is literally spe...,0,0,0,0,0,0.0
196,87156,You telling me that I gotta play the fucking D...,ɴᴏᴡ ᴘʟᴀʏɪɴɢ: [Rage Against The Machine - B](ht...,YEEEEEESSSSSSS,0,0,0,0,0,0.0
197,88708,UWMC eod $14,Don’t think I’ll be able to rub one out today,Fun fact: right now QQQ is only up about 2% ov...,0,0,0,0,0,0.0
198,94490,CLNE somehow still up today,"Started at 6k, now at 300.",Mmmmmm... never-ending dip...,0,0,0,0,0,0.0


In [31]:
selected_columns = ['example_id', 'preceding_comment', 'comment_for_evaluation',
       'following_comment', 'toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']

list_of_labelled_data = [pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] for fname in labelled_data_fnames]

In [22]:

selected_columns = ['example_id', 'preceding_comment', 'comment_for_evaluation',
       'following_comment', 'toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']

list_of_labelled_data = [pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] for fname in labelled_data_fnames]

KeyError: "['comment_for_evaluation'] not in index"

In [32]:
labelled_master_data = pd.concat(list_of_labelled_data )

In [33]:
labelled_master_data.to_csv(master_data_location,index=False)

In [34]:
labelled_master_data.head(5)

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat
0,104857,[https://www.youtube.com/watch?v=hRBOnA0ak4w&a...,GME just watching this all and doing nothing lol,[https://www.youtube.com/watch?v=hRBOnA0ak4w&a...,0.0,0.0,0.0,0.0,0.0,0.0
1,105125,#GOTTA GET THAT 1.48% ON A BOND WHOOOO,"Everything is down today, knee jerk reaction t...","AMC, I held all this time and I decided to buy...",0.0,0.0,0.0,0.0,0.0,0.0
2,105684,Having 30k to lose sounds pretty nice to me,"No, this is the Warthog",a good trump tweet would turn all this carnage...,0.0,0.0,0.0,0.0,0.0,0.0
3,105904,His tools for inflation are literally rates an...,The market is not worth what you guys have bee...,Can somebody make clear that it's not us who a...,0.0,0.0,0.0,0.0,0.0,0.0
4,1138,Wot?,Didnt he sell all his shares?,This is the way lol holding 100 March 12 / $51...,0.0,0.0,0.0,0.0,0.0,0.0
